# Alpha Sherpa Capital Research
PARK, Jung Jun David
### Library Version:
* pandas : 1.0.1
* numpy  : 1.19.5

In [1]:
# Importing Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import numpy as np
import FinanceDataReader as fdr
from datetime import datetime, timedelta
from pandas import Timestamp

### Loading CSV File from Benzinga
* https://www.benzinga.com/analyst-ratings


In [2]:
raw_df = pd.read_csv('/Users/david/Documents/Analyst_Rating.csv', header = 0, index_col = 0).reset_index()

In [3]:
raw_df

,Date,Ticker,Company,Upside/Downside,Analyst Firm,Price Target Change,Rating Change,Previous / Current Rating
0,2021-03-12,NLSP,NLS Pharmaceutics,105.13,Brookline Capital,12.0,Initiates Coverage On,Buy
1,2021-03-12,INSW,International Seaways,33.06,Jefferies,26.0,Maintains,Buy
2,2021-03-12,DSSI,Diamond S Shipping,29.03,Jefferies,12.0,Maintains,Buy
3,2021-03-12,JD,JD.com,28.54,HSBC,108.0,Maintains,Buy
4,2021-03-12,POSH,Poshmark,74.26,MKM Partners,83.0,Maintains,Buy
...,...,...,...,...,...,...,...,...
644,2021-03-08,CLF,Cleveland-Cliffs,13.96,KeyBanc,20.0,Maintains,Overweight
645,2021-03-08,TTD,Trade Desk,13.28,KeyBanc,851.0,Upgrades,Overweight
646,2021-03-08,PRAH,PRA Health Sciences,NaN,KeyBanc,NaN,Downgrades,Sector Weight
647,2021-03-08,IQV,IQVIA Holdings,16.58,KeyBanc,218.0,Upgrades,Overweight


### Deleting Coverage Initiation as well as Maintain Rating

In [4]:
raw_df = raw_df.drop(raw_df[raw_df['Rating Change'] == 'Initiates Coverage On'].index)
for i in range(len(raw_df)):
    if (raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] == 'Assumes'
       or raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] == 'Reiterates'):
        raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] = 'Maintains'
        
    elif (raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] == 'Upgrades'):
        raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] = 'Up'
    
    elif (raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] == 'Downgrades'):
        raw_df.iloc[i,raw_df.columns.get_loc('Rating Change')] = 'Down'

raw_df = raw_df.drop(raw_df[raw_df['Rating Change'] == 'Maintains'].index)

### Count by Rating Changes

In [5]:
raw_df.groupby('Rating Change').count()

,Date,Ticker,Company,Upside/Downside,Analyst Firm,Price Target Change,Previous / Current Rating
Rating Change,,,,,,,
Down,96,96,96,64,96,64,96
Up,110,110,110,93,110,93,110


### Initializing DataFrame as desired

In [62]:
df = pd.DataFrame(index = raw_df.index,columns = {'Date', 'Ticker', 'Company', 'Upside/Downside', 'Analyst Firm',
       'Price Target Change', 'Rating Change', 'Previous / Current Rating', '2021-03-08 Open', 
        '2021-03-08 Close', '2021-03-09 Open', '2021-03-09 Close','2021-03-10 Open', '2021-03-10 Close',
        '2021-03-11 Open', '2021-03-11 Close','2021-03-12 Open', '2021-03-12 Close'})
df = df[['Date', 'Ticker', 'Company', 'Upside/Downside', 'Analyst Firm',
       'Price Target Change', 'Rating Change', 'Previous / Current Rating', '2021-03-08 Open', 
        '2021-03-08 Close', '2021-03-09 Open', '2021-03-09 Close','2021-03-10 Open', '2021-03-10 Close',
        '2021-03-11 Open', '2021-03-11 Close','2021-03-12 Open', '2021-03-12 Close']]

for i in range(len(raw_df)):
    for j in range(len(raw_df.columns)):
        df.iat[i,j] = raw_df.iloc[i,j]

df

,Date,Ticker,Company,Upside/Downside,Analyst Firm,Price Target Change,Rating Change,Previous / Current Rating,2021-03-08 Open,2021-03-08 Close,2021-03-09 Open,2021-03-09 Close,2021-03-10 Open,2021-03-10 Close,2021-03-11 Open,2021-03-11 Close,2021-03-12 Open,2021-03-12 Close
13,2021-03-12,SYN,Synthetic Biologics,NaN,Maxim Group,NaN,Up,Buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2021-03-12,KDP,Keurig Dr Pepper,NaN,Morgan Stanley,NaN,Down,Equal-Weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2021-03-12,BMTC,Bryn Mawr Bank,-11.62,"Keefe, Bruyette & Woods",42,Down,Market Perform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2021-03-12,BOSSY,Hugo Boss,NaN,Societe Generale,NaN,Down,Sell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,2021-03-12,BMWYY,BMW,NaN,Barclays,NaN,Down,Equal-Weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,2021-03-08,ICLR,Icon,17.95,Baird,215,Up,Outperform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,2021-03-08,TTD,Trade Desk,13.28,KeyBanc,851,Up,Overweight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,2021-03-08,PRAH,PRA Health Sciences,NaN,KeyBanc,NaN,Down,Sector Weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,2021-03-08,IQV,IQVIA Holdings,16.58,KeyBanc,218,Up,Overweight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Downloading Price Data from Last Week (2021-02-08 ~ 2021-03-12)
* Extra data for ATR Calculations

In [7]:
ticker_list = list( dict.fromkeys(df['Ticker']))
open_data_list = [fdr.DataReader(ticker, '2021-02-08','2021-03-12')['Open'] for ticker in ticker_list]
open_data = pd.concat(open_data_list, axis = 1)
open_data.columns = [ticker for ticker in ticker_list] 

close_data_list = [fdr.DataReader(ticker, '2021-02-08','2021-03-12')['Close'] for ticker in ticker_list]
close_data = pd.concat(close_data_list, axis = 1)
close_data.columns = [ticker for ticker in ticker_list] 

high_data_list = [fdr.DataReader(ticker, '2021-02-08','2021-03-12')['High'] for ticker in ticker_list]
high_data = pd.concat(high_data_list, axis = 1)
high_data.columns = [ticker for ticker in ticker_list] 

low_data_list = [fdr.DataReader(ticker, '2021-02-08','2021-03-12')['Low'] for ticker in ticker_list]
low_data = pd.concat(low_data_list, axis = 1)
low_data.columns = [ticker for ticker in ticker_list] 

### Filling Open/Close Price to the DataFrame

In [103]:
for i in range(len(df)):
    ticker = df.iloc[i,df.columns.get_loc('Ticker')]
    for j in range(8,13):
        if (j< 10):
            day = '2021-03-0'+str(j)
        else:
            day = '2021-03-' +str(j)
        
        df.iat[i,df.columns.get_loc(day+' Open')] = open_data.iat[open_data.index.get_loc(day),open_data.columns.get_loc(ticker)]
        df.iat[i,df.columns.get_loc(day+' Close')] = close_data.iat[close_data.index.get_loc(day),open_data.columns.get_loc(ticker)]

## Initial Look at the Data
### Announcement Day Trade Results / Market Reaction / Impact
* How did the stock move on the day, analyst made the announcement?
* How did the Market React? Did stock price rise if it was a raise and vice versa?
* If Stock Price rose with Analyst's raise, then we consider the Impact to be True.
* Here we only look at brokers that made more than 5 announcements.

In [64]:
# 1 day trading results
df['1 Day Trade'] = 0.00
df['Market Reaction'] = 'Down'
df['Impact'] = False
for i in range(len(df)):
    start_date = df.iloc[i,0]
    trade_open = df.iat[i,df.columns.get_loc(start_date + ' Open')]
    trade_close = df.iat[i,df.columns.get_loc(start_date + ' Close')]
    df.iat[i,df.columns.get_loc('1 Day Trade')] = (trade_close - trade_open) / trade_open * 100
    if (df.iat[i,df.columns.get_loc('1 Day Trade')] > 0):
        df.iat[i,df.columns.get_loc('Market Reaction')] = 'Up'
    
    if ((df.iat[i,df.columns.get_loc('Rating Change')] == 'Up') and (df.iat[i,df.columns.get_loc('Market Reaction')] == 'Up')):
        df.iat[i,df.columns.get_loc('Impact')] = True
    
    elif ((df.iat[i,df.columns.get_loc('Rating Change')] == 'Down') and (df.iat[i,df.columns.get_loc('Market Reaction')] == 'Down')):
        df.iat[i,df.columns.get_loc('Impact')] = True
        
df = df.set_index('Date')

### Number of Long/Short
* As I will strictly follow the rating changes made by analysts, Up represents Long and Down represents Short

In [66]:
df.groupby('Rating Change')['Ticker'].count()

Rating Change
Down     96
Up      110
Name: Ticker, dtype: int64

### Broker's Impact
* Impact is measured by how much the market followed the analysts' calls.
* For the sake of larger dataset, I used 1 day data for this.
* Only looked at firms that made over 5 rating changes as less would dilute the result.

In [104]:
num_inline = df.groupby(['Analyst Firm']).apply(lambda x: (x['Impact'] == True).sum())
num_total_announcements = df.groupby(['Analyst Firm'])['Ticker'].count()
num_over5_announcements = num_total_announcements[num_total_announcements >= 5]

broker_impact = pd.DataFrame(index = num_over5_announcements.index, columns = {'Impact Rate'})
for i in range(len(num_over5_announcements)):
    for j in range(len(num_total_announcements)):
        if (num_over5_announcements.index[i] == num_inline.index[j]):
            broker_impact.iloc[i,0] = num_inline[j]/num_over5_announcements[i]
    
broker_impact.sort_values(by = 'Impact Rate', ascending = False)

,Impact Rate
Analyst Firm,
Barclays,0.714286
Johnson Rice,0.666667
KeyBanc,0.666667
Piper Sandler,0.666667
Raymond James,0.666667
Deutsche Bank,0.625
BMO Capital,0.6
Oppenheimer,0.6
Wells Fargo,0.555556


* From this graph, we could conclude that Barclays, Johnson Rice, KeyBanc are the strongest brokers.
* JP Morgan and UBS are the weakest broker.
* Will not conduct a further reseach on how much profit could have been earned with strong brokers as that would be redundant.

## Continuing with the Research Task
### Pre-processing Functions

In [74]:
def true_range(high_df, low_df, close_df):
    #TR = max( H - L , abs(High - yesterday's Close), abs(yesterday's Close - Low) )
    #ATR = 1/n <(TR)
    
    TR_df = pd.DataFrame(index = close_df.index, columns = close_df.columns)
    for column_idx in range(len(close_df.columns)):
        for row_idx in range(len(close_df)):
            if (row_idx == 0):
                true_range = 0
            else:
                high_low = high_df.iloc[row_idx,column_idx] - low_df.iloc[row_idx,column_idx]
                high_y_close = abs(high_df.iloc[row_idx,column_idx] - close_df.iloc[row_idx - 1,column_idx])
                low_y_close = abs(low_df.iloc[row_idx,column_idx] - close_df.iloc[row_idx - 1,column_idx])
                true_range = max(high_low, high_y_close, low_y_close)
            
            TR_df.iat[row_idx,column_idx] = true_range
          
    return TR_df


def average_true_range(true_range):
    ATR_df = pd.DataFrame(index = true_range.index, columns = true_range.columns)
    for column_idx in range(len(true_range.columns)):
        for row_idx in range(len(true_range)):
            if (row_idx == 0):
                avg_true_range = 0
            else:
                avg_true_range = ATR_df.iat[row_idx-1,column_idx] * 9 + true_range.iat[row_idx,column_idx]
                avg_true_range = avg_true_range / 10
            
            ATR_df.iat[row_idx,column_idx] = avg_true_range
  
    return ATR_df

def calculate_performance(original_df, open_df, close_df, ATR_df):
    perf_df = pd.DataFrame(index = {'Day 1','Day 2','Day 3','Day 4','Day 5','Suggestion','Firm'},
                           columns = ATR_df.columns).sort_index()
    
    for col_idx in range(len(perf_df.columns)): #Ticker
        ticker = ATR_df.columns[col_idx]
        entry_date = original_df[original_df['Ticker'] == ticker].index[0]
        analyst_firm = original_df[original_df['Ticker'] == ticker]['Analyst Firm'][0]
        suggestion = original_df[original_df['Ticker'] == ticker]['Rating Change'][0]
        
        perf_df.iloc[perf_df.index.get_loc('Suggestion'), col_idx] = suggestion
        perf_df.iloc[perf_df.index.get_loc('Firm'), col_idx] = analyst_firm
        
        
        original_df = original_df[1:]   # Get rid of this row now
        entry_date2 = original_df[original_df['Ticker'] == ticker].index
        exit_date = entry_date
        entry_price = open_df.loc[entry_date , ticker]
        exit_price = close_df.loc[exit_date, ticker]
        compare_open_date = datetime.strptime(entry_date, '%Y-%m-%d')
        
        ATR = ATR_df.iloc[ATR_df.index.get_loc(entry_date), ATR_df.columns.get_loc(ticker)]
        
        if (suggestion == 'Up'):
            perf_df.iat[0, col_idx] = ( exit_price - entry_price) / ATR
            
        else:
            perf_df.iat[0, col_idx] = ( entry_price - exit_price) / ATR
        
        compare_close_date = datetime.strptime(exit_date, '%Y-%m-%d')
        last_day = datetime.strptime('2021-03-12','%Y-%m-%d')
        
        count = 0
        while ( compare_close_date < last_day):
            count += 1
            compare_close_date += timedelta(days=1)
            exit_date = datetime.strftime(compare_close_date, '%Y-%m-%d')
            exit_price = close_df.loc[exit_date, ticker]
            
            if (suggestion == 'Up'):
                perf_df.iat[count, col_idx] = (exit_price - entry_price) / ATR
            
            else:
                perf_df.iat[count, col_idx] = (entry_price - exit_price) / ATR

    return perf_df
    

In [105]:
TR_df = true_range(high_data, low_data, close_data)
ATR_df = average_true_range(TR_df).dropna(axis =1)  # Dropped One column with Nan value from Crawling
ATR_df

,SYN,KDP,BMTC,BMWYY,VLDR,MMP,UBER,BCC,MDC,PTVE,...,PTON,IOVA,VTRS,ETN,KO,ICLR,TTD,PRAH,IQV,ETR
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-02-09,0.018,0.063,0.114,0.04,0.131,0.103,0.244,0.154,0.159,0.07,...,0.49,0.232,0.069,0.201,0.039,0.395,3.02,0.247,0.335,0.15
2021-02-10,0.0462,0.1297,0.2086,0.069,0.3679,0.1797,0.6086,0.2406,0.3361,0.178,...,1.136,0.5968,0.1071,0.4769,0.1401,1.2525,6.342,0.7613,1.9185,0.291
2021-02-11,0.05558,0.16773,0.29274,0.1211,0.46411,0.23473,0.91274,0.44054,0.49449,0.2622,...,1.4114,0.74612,0.13139,0.68121,0.22509,1.52525,8.0478,1.08717,2.41765,0.4349
2021-02-12,0.058022,0.188957,0.312466,0.14399,0.502699,0.337257,0.983466,0.563486,0.574041,0.32098,...,2.26526,0.835508,0.155251,0.801089,0.257581,1.85673,8.94402,1.41745,2.60789,0.50241
2021-02-16,0.0577198,0.223061,0.373219,0.152591,0.653429,0.386531,1.03212,0.745137,0.836637,0.329882,...,3.01573,0.936957,0.168726,0.95998,0.287823,2.36205,11.1496,1.67071,2.6961,0.583169
2021-02-17,0.0639278,0.240755,0.417897,0.168332,0.704086,0.433878,1.12891,0.859624,0.880973,0.334894,...,3.79916,1.13626,0.188853,1.06798,0.320041,2.77485,15.2307,1.84964,2.77149,0.690852
2021-02-18,0.071435,0.26868,0.433108,0.187499,0.753678,0.53749,1.27202,0.939661,0.885876,0.330404,...,3.95324,1.37464,0.209968,1.11518,0.368037,3.08936,16.1826,1.95967,2.85934,0.754767
2021-02-19,0.0740715,0.304812,0.483797,0.203749,0.82531,0.621741,1.37982,1.1237,0.924288,0.393364,...,3.93992,1.36917,0.220971,1.35067,0.417233,3.51843,23.0763,2.02771,2.9214,0.80929


### Performance Calculation
* Performance Measure is (Exit - Entry) / ATR  (opposite for short)
* For ATR, I used the ATR from the day before. Regardless of the holding period.
* Since some of these announcements were made on 03/12, can only do 1 day trading.

In [76]:
perf_df = calculate_performance(df, open_data, close_data, ATR_df)
perf_df = perf_df.transpose()
perf_df

,Day 1,Day 2,Day 3,Day 4,Day 5,Firm,Suggestion
SYN,1.15983,NaN,NaN,NaN,NaN,Maxim Group,Up
KDP,-0.113373,NaN,NaN,NaN,NaN,Morgan Stanley,Down
BMTC,0.15312,NaN,NaN,NaN,NaN,"Keefe, Bruyette & Woods",Down
BMWYY,-0.561994,NaN,NaN,NaN,NaN,Barclays,Down
VLDR,-0.356985,NaN,NaN,NaN,NaN,Berenberg,Down
...,...,...,...,...,...,...,...
ICLR,0.0269926,0.143061,-0.373848,0.0904254,0.305017,Baird,Up
TTD,0.000362359,0.731966,0.791574,1.40559,1.30993,KeyBanc,Up
PRAH,0.311449,-0.319924,0.0169496,-0.122885,-0.362298,KeyBanc,Down
IQV,-0.313627,0.0122991,0.0840439,0.00819941,-0.516563,KeyBanc,Up


In [77]:
perf_df[['Day 1','Day 2','Day 3', 'Day 4', 'Day 5']].sum()

Day 1    10.232362
Day 2    12.509923
Day 3    14.701856
Day 4    33.054190
Day 5    26.612322
dtype: float64

* Right away, we can tell that such a strategy could be very effective given the R values returned.
* It is evident that despite having less equities, as the # of days increase, performance increase.
* Day 4 seems to have the highest increase with highest momentum.
* Afterwards, the momentum starts to slow down and show adjustments.

### Expectancy
* Risk will be set at USD 1000 per trade.
* Will trade intraday (Day 1 Open and Close).
* Barclays, Johnson Rice, KeyBanc for Expectancy.

In [107]:
Risk = 1000
BAC_r = perf_df[perf_df['Firm'] == 'Barclays'].groupby('Firm')['Day 1'].sum()
JR_r = perf_df[perf_df['Firm'] == 'Johnson Rice'].groupby('Firm')['Day 1'].sum()
KB_r = perf_df[perf_df['Firm'] == 'KeyBanc'].groupby('Firm')['Day 1'].sum()

BAC_hit = broker_impact.sort_values(by = 'Impact Rate', ascending = False)['Impact Rate'][0]
JR_hit = broker_impact.sort_values(by = 'Impact Rate', ascending = False)['Impact Rate'][1]
KB_hit = broker_impact.sort_values(by = 'Impact Rate', ascending = False)['Impact Rate'][2]

BAC_exp = Risk * BAC_r[0]  * BAC_hit
JR_exp = Risk * JR_r[0]  * JR_hit
KB_exp = Risk * KB_r[0]  * KB_hit

print("Expectancy with Risk of US$1000/trade with the three brokers is USD",round(BAC_exp + JR_exp + KB_exp,2))

Expectancy with Risk of US$1000/trade with the three brokers is USD 1318.04
